# Notebook for ingesting data into azure data explorer cluster

In [2]:
import pandas as pd
import plotly.express as px
from matplotlib import pyplot as plt
from scipy import signal 
from ae_adx_client.ae_adx_client import AdxClient
import os

In [24]:
input_data_2_df = pd.read_parquet('../data/input_dataset-2.parquet')
input_data_2_df.head()

,Unit_4_Power,Unit_4_Reactive Power,Turbine_Guide Vane Opening,Turbine_Pressure Drafttube,Turbine_Pressure Spiral Casing,Turbine_Rotational Speed,mode,Bolt_1_Steel tmp,Bolt_1_Tensile,Bolt_2_Tensile,...,Bolt_5_Tensile,Bolt_6_Tensile,Bolt_1_Torsion,Bolt_2_Torsion,Bolt_3_Torsion,Bolt_4_Torsion,Bolt_5_Torsion,Bolt_6_Torsion,lower_bearing_vib_vrt,turbine_bearing_vib_vrt
timepoints,,,,,,,,,,,,,,,,,,,,,
1970-12-19 09:51:44,262.204308,2.899036,82.279976,173.955216,5310.799181,107.964278,operation,4.133996,1598.481390,1480.989917,...,1635.585700,1674.848803,175.758460,163.956613,146.288741,225.535170,297.780208,161.148100,NaN,NaN
1970-12-19 09:51:45,262.104319,3.344630,82.277248,173.989815,5311.219755,107.964273,operation,4.134078,1598.477449,1480.989528,...,1635.588478,1674.823883,175.755164,163.951680,146.284164,225.527142,297.771627,161.145094,NaN,NaN
1970-12-19 09:51:46,262.004330,3.790223,82.274520,174.024413,5311.640329,107.964269,operation,4.134731,1598.479316,1481.003188,...,1635.583464,1674.841318,175.764601,163.952007,146.283423,225.522291,297.777115,161.144487,NaN,NaN
1970-12-19 09:51:47,261.904340,4.235817,82.271792,174.059012,5312.060902,107.964264,operation,4.134270,1598.490184,1481.028827,...,1635.581384,1674.843245,175.763157,163.953924,146.283633,225.535827,297.772578,161.144037,NaN,NaN
1970-12-19 09:51:48,261.804351,4.064759,82.269064,174.153819,5312.405938,107.964259,operation,4.133583,1598.494073,1481.059017,...,1635.591746,1674.872300,175.760959,163.951968,146.286946,225.534231,297.774191,161.151967,NaN,NaN


In [3]:
input_data_1_df = pd.read_parquet('../data/input_dataset-1.parquet')

In [4]:
input_data_1_df.head()

,Unit_4_Power,Unit_4_Reactive Power,Turbine_Guide Vane Opening,Turbine_Rotational Speed,Turbine_Pressure Drafttube,Turbine_Pressure Spiral Casing,Bolt_1_Steel tmp,Bolt_1_Torsion,Bolt_1_Tensile,Bolt_2_Torsion,Bolt_2_Tensile,Bolt_3_Torsion,Bolt_3_Tensile,Bolt_4_Torsion,Bolt_4_Tensile,Bolt_5_Torsion,Bolt_5_Tensile,Bolt_6_Torsion,Bolt_6_Tensile,mode
timepoints,,,,,,,,,,,,,,,,,,,,
1970-06-24 09:48:36,0.0,-1.407876,1.262137,0.984012,214.159091,205.055479,7.455880,155.148968,1405.125823,130.792481,1364.298361,161.795317,1545.338423,214.092594,1541.879099,270.753014,1523.484042,150.872083,1503.087832,start
1970-06-24 09:48:37,0.0,-1.425341,1.266306,0.987262,214.147845,205.039166,7.456339,155.699001,1414.472359,130.446843,1369.287791,161.794785,1549.648881,214.436328,1547.591601,271.097226,1531.876456,150.595804,1518.719841,start
1970-06-24 09:48:38,0.0,-1.442806,1.270476,0.990513,214.172126,205.071560,7.454355,155.789993,1416.526738,130.375745,1370.435669,161.833483,1550.420271,214.488576,1548.632153,271.143303,1533.457978,150.547930,1522.138148,start
1970-06-24 09:48:39,0.0,-1.460271,1.274646,0.993764,214.201458,205.110880,7.452470,155.833039,1417.425442,130.356274,1370.953391,161.811008,1550.729481,214.501790,1549.060964,271.155305,1534.171959,150.526966,1523.589704,start
1970-06-24 09:48:40,0.0,-1.477737,1.278815,0.997015,214.230791,205.150199,7.452556,155.875327,1418.311627,130.329982,1371.439562,161.804882,1551.064989,214.526694,1549.477768,271.191701,1534.817494,150.507935,1524.958113,start


In [5]:
data_1_pivot = input_data_1_df.melt(ignore_index=False)

In [6]:
data_1_pivot.columns = ['sensor','value']
data_1_pivot.sort_index(inplace=True)
data_1_pivot.head()

,sensor,value
timepoints,,
1970-06-24 09:48:36,Unit_4_Power,0.0
1970-06-24 09:48:36,Bolt_4_Torsion,214.092594
1970-06-24 09:48:36,Bolt_2_Tensile,1364.298361
1970-06-24 09:48:36,Bolt_4_Tensile,1541.879099
1970-06-24 09:48:36,Bolt_2_Torsion,130.792481


In [7]:
adx_client = AdxClient.from_log_on(adx_cluster_url=os.getenv('adxprdcluster'),database='TestDB')
adx_client.tables

,TableType,DocString,SourceTable
TableName,,,
ScadaTest,Source,None,NaN
HendelserTest,Source,None,NaN
TestIngest,Source,None,NaN
Input_2,Source,None,NaN
Input_1,Source,None,NaN
prediction_data,Source,None,NaN


In [40]:
adx_client.table_schemas['Input_1']

{'timepoints': 'System.DateTime',
 'sensor': 'System.String',
 'value': 'System.Double'}

In [11]:
for i in range(0,len(data_1_pivot),100000):
    
    end_index = i + 100000
    print(i,end_index)
    if end_index<len(data_1_pivot):
        adx_client.ingest_data_frame(df= data_1_pivot.iloc[i:end_index,:],target_table='Input_1')

0 100000
100000 200000
200000 300000
300000 400000
400000 500000
500000 600000
600000 700000
700000 800000
800000 900000
900000 1000000
1000000 1100000
1100000 1200000
1200000 1300000
1300000 1400000
1400000 1500000
1500000 1600000
1600000 1700000
1700000 1800000
1800000 1900000
1900000 2000000
2000000 2100000
2100000 2200000
2200000 2300000
2300000 2400000
2400000 2500000
2500000 2600000
2600000 2700000
2700000 2800000
2800000 2900000
2900000 3000000
3000000 3100000
3100000 3200000
3200000 3300000
3300000 3400000
3400000 3500000
3500000 3600000
3600000 3700000
3700000 3800000
3800000 3900000
3900000 4000000
4000000 4100000
4100000 4200000
4200000 4300000
4300000 4400000
4400000 4500000
4500000 4600000
4600000 4700000
4700000 4800000
4800000 4900000
4900000 5000000
5000000 5100000
5100000 5200000
5200000 5300000
5300000 5400000
5400000 5500000
5500000 5600000
5600000 5700000
5700000 5800000
5800000 5900000
5900000 6000000
6000000 6100000
6100000 6200000
6200000 6300000
6300000 6400000
